In [ ]:


#movie rating
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')


In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10329 entries, 0 to 10328
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  10329 non-null  int64 
 1   title    10329 non-null  object
 2   genres   10329 non-null  object
dtypes: int64(1), object(2)
memory usage: 242.2+ KB


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105339 entries, 0 to 105338
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     105339 non-null  int64  
 1   movieId    105339 non-null  int64  
 2   rating     105339 non-null  float64
 3   timestamp  105339 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.2 MB


In [ ]:
movies.describe()

,movieId
count,10329.000000
mean,31924.282893
std,37734.741149
min,1.000000
25%,3240.000000
50%,7088.000000
75%,59900.000000
max,149532.000000


In [ ]:
ratings.describe()

,userId,movieId,rating,timestamp
count,105339.000000,105339.000000,105339.000000,1.053390e+05
mean,364.924539,13381.312477,3.516850,1.130424e+09
std,197.486905,26170.456869,1.044872,1.802660e+08
min,1.000000,1.000000,0.500000,8.285650e+08
25%,192.000000,1073.000000,3.000000,9.711008e+08
50%,383.000000,2497.000000,3.500000,1.115154e+09
75%,557.000000,5991.000000,4.000000,1.275496e+09
max,668.000000,149532.000000,5.000000,1.452405e+09


In [ ]:
#average rating of movie is 3.5
#finding unique users and movies

movies['movieId'].nunique(),ratings['movieId'].nunique()

(10329, 10325)

In [ ]:
ratings['userId'].nunique()

668

In [ ]:
#668 unique users ,rating 10325



In [ ]:
movies['genres'] = movies['genres'].astype(str)

In [ ]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'].head()

0    [Adventure, Animation, Children, Comedy, Fantasy]
1                       [Adventure, Children, Fantasy]
2                                    [Comedy, Romance]
3                             [Comedy, Drama, Romance]
4                                             [Comedy]
Name: genres, dtype: object

In [ ]:
movies2=movies.explode('genres')
movies2.head(20)

,movieId,title,genres
0,1,Toy Story (1995),Adventure
0,1,Toy Story (1995),Animation
0,1,Toy Story (1995),Children
0,1,Toy Story (1995),Comedy
0,1,Toy Story (1995),Fantasy
1,2,Jumanji (1995),Adventure
1,2,Jumanji (1995),Children
1,2,Jumanji (1995),Fantasy
2,3,Grumpier Old Men (1995),Comedy
2,3,Grumpier Old Men (1995),Romance


In [ ]:
#mean rating
merged_info=pd.merge(movies2,ratings,on=['movieId'],how='inner')
merged_info

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure,2,5.0,859046895
1,1,Toy Story (1995),Adventure,5,4.0,1303501039
2,1,Toy Story (1995),Adventure,8,5.0,858610933
3,1,Toy Story (1995),Adventure,11,4.0,850815810
4,1,Toy Story (1995),Adventure,14,4.0,851766286
...,...,...,...,...,...,...
281899,148238,A Very Murray Christmas (2015),Comedy,475,3.0,1451213043
281900,148626,The Big Short (2015),Drama,458,4.0,1452014749
281901,148626,The Big Short (2015),Drama,576,4.5,1451687664
281902,148626,The Big Short (2015),Drama,668,4.5,1451148148


In [ ]:

print(merged_info.groupby('genres').agg({'title':'nunique','rating':'mean'}).reset_index().rename(columns={'title':'unique_movies_count','rating':'mean_rating'}))

                genres  unique_movies_count  mean_rating
0   (no genres listed)                    7     3.071429
1               Action                 1736     3.451450
2            Adventure                 1164     3.518027
3            Animation                  400     3.635350
4             Children                  540     3.439429
5               Comedy                 3513     3.420996
6                Crime                 1440     3.642392
7          Documentary                  415     3.643035
8                Drama                 5217     3.650266
9              Fantasy                  670     3.500459
10           Film-Noir                  195     3.913636
11              Horror                 1001     3.281097
12                IMAX                  152     3.641821
13             Musical                  409     3.571962
14             Mystery                  675     3.652043
15             Romance                 1788     3.544255
16              Sci-Fi         

In [ ]:
movies2['genres'].unique()

array(['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Mystery', 'Sci-Fi', 'IMAX', 'War', 'Musical', 'Documentary',
       'Western', 'Film-Noir', '(no genres listed)'], dtype=object)

In [ ]:
movies2['genres'].nunique()

20

In [ ]:
#there are 20 but one is no genres listed hence 19
#remove it


In [ ]:
movies2=movies2[movies2['genres']!='(no genres listed)']
movies2['genres'].nunique()


19

In [ ]:
#popularity based recommendations
#user defined function to display topN movies to recommended
def TopNPopularMovies(genre,num_ratings_threshold,topN=5):
  #calculating mean of ratings and number of ratings
  popularity_df=merged_info.groupby(['genres','title']).agg({'rating':['mean','size']}).reset_index()
  popularity_df.columns=['genre','title','ratings_mean','ratings_counts']
#filtering desired genre and number of reviews threshold
  topN_recommendations=popularity_df[(popularity_df['genre']==genre)&(popularity_df['ratings_counts']>=num_ratings_threshold)].sort_values(by=['ratings_mean'],ascending=False).head(topN)
#refactoring output
  topN_recommendations['S.No']=list(range(1,len(topN_recommendations)+1))
  topN_recommendations.index=range(len(topN_recommendations))
  topN_recommendations.columns=['Genre','Movie Title','Average Movie Rating','Num Reviews','S.No']
  return topN_recommendations[['S.No','Movie Title','Average Movie Rating','Num Reviews']]

In [ ]:
#testing function :try1
genre='Action'
num_ratings_threshold=100
topN=10
TopNPopularMovies(genre=genre,
                  num_ratings_threshold=num_ratings_threshold,
                  topN=topN)


,S.No,Movie Title,Average Movie Rating,Num Reviews
0,1,"Matrix, The (1999)",4.264368,261
1,2,Star Wars: Episode V - The Empire Strikes Back...,4.228070,228
2,3,Raiders of the Lost Ark (Indiana Jones and the...,4.212054,224
3,4,Inception (2010),4.189320,103
4,5,Star Wars: Episode IV - A New Hope (1977),4.188645,273
5,6,Fight Club (1999),4.188406,207
6,7,Blade Runner (1982),4.169872,156
7,8,"Princess Bride, The (1987)",4.163743,171
8,9,Aliens (1986),4.146497,157
9,10,"Dark Knight, The (2008)",4.141732,127


In [ ]:
genre='Animation'
num_ratings_threshold=100
topN=10
TopNPopularMovies(genre=genre,
                  num_ratings_threshold=num_ratings_threshold,
                  topN=topN)


,S.No,Movie Title,Average Movie Rating,Num Reviews
0,1,"Incredibles, The (2004)",3.926471,102
1,2,Toy Story 2 (1999),3.913462,104
2,3,Toy Story (1995),3.907328,232
3,4,Finding Nemo (2003),3.888889,117
4,5,"Monsters, Inc. (2001)",3.879310,116
5,6,"Lion King, The (1994)",3.770950,179
6,7,Shrek (2001),3.756329,158
7,8,"Bug's Life, A (1998)",3.607843,102
8,9,Aladdin (1992),3.602094,191
9,10,Beauty and the Beast (1991),3.576923,143


In [ ]:
genre='Comedy'
num_ratings_threshold=50
topN=5
TopNPopularMovies(genre=genre,
                  num_ratings_threshold=num_ratings_threshold,
                  topN=topN)

,S.No,Movie Title,Average Movie Rating,Num Reviews
0,1,Monty Python and the Holy Grail (1975),4.301948,154
1,2,Fargo (1996),4.271144,201
2,3,Life Is Beautiful (La Vita è bella) (1997),4.253425,73
3,4,"Sting, The (1973)",4.207792,77
4,5,Annie Hall (1977),4.205882,68
